In [ ]:
import cv2
import numpy as np
import time
import matplotlib.pyplot as plt

from skimage.io import imread, imshow


from skimage import img_as_ubyte, img_as_float
from scipy.stats import norm

from IPython.display import clear_output
face_cascade = cv2.CascadeClassifier("C:\\Users\\Dell\\haarcascade_frontalface_default.xml")

In [ ]:
face_cascade.empty()

In [ ]:
c = int(input("Enter Webcam no: "))


capture=cv2.VideoCapture(c)

cod = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('out.avi',cod,20.0,(640,480))
f = 0
img = []
drtn = 0
while True:
    if(f == 0):
        time.sleep(2)
        start = time.time()
    ret,color_img =capture.read()
    cv2.imshow('Frames',color_img)

    gray_img = cv2.cvtColor(color_img, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(gray_img)
    
    if(len(faces) >1):
        continue
        
    for (x,y,w,h) in faces:
        face_gray = gray_img[y:y+h, x:x+h]
        face_color = color_img[y:y+h, x:x+w]
        
        h = h//3 - 15
        x += 15
        w -= 38
        img = color_img[y:y+h,x:x+w]
    if(len(img) == 0):
        continue
    cv2.imshow('img',img)

    cv2.imwrite('Frame'+str(f)+'.jpg', img)
    f += 1

    if cv2.waitKey(1) & 0xFF == ord('q'):
        drtn = (time.time() - start)
        break
        
capture.release()
cv2.destroyAllWindows()

In [ ]:
f= 246
drtn = 8
f//drtn

In [ ]:
import os
bgr = np.zeros((f,4))
R = []
G = []
B = []

for k in range(f):

        
    img = cv2.imread('C:\\Users\\Dell\\IPPG PROJECT\\Frame'+str(k)+'.jpg')  #Start from number 1
    #print('frame'+str(k)+'.jpg')
    #clear_output(wait=True)
    #try:
  #os.remove('C:\\Users\\Dell\\Frame'+str(k)+'.jpg')
   # except:
    #     break
            
    h, w, c = img.shape #height, width, channnel

    #Initialize
    l=0
    b_ave=0; g_ave=0; r_ave=0

    rCh = img[:,:,0]
    gCh = img[:,:,1]
    bCh = img[:,:,2]
    
    R.append(np.mean(rCh))
    G.append(np.mean(gCh))
    B.append(np.mean(bCh))

In [ ]:
R = R[:len(R)-300]   # Lower the sample rate from backside
G = G[:len(G)-300]
B = B[:len(B)-300]

In [ ]:
from scipy import signal
resample_R = signal.resample(R, 1744)   # resample the data (increase or decrease)
resample_G = signal.resample(G, 1744)
resample_B = signal.resample(B, 1744)

In [ ]:
R1 = []
G1 = []
B1 = []
for i in range(len(R)):     # appending None value in raw data
    R1.append(R[i])
    R1.append(None)
    
    G1.append(G[i])
    G1.append(None)
    
    B1.append(B[i])
    B1.append(None)
        

R = np.array(R1)
G = np.array(G1)
B = np.array(B1)

import pandas as pd
dataframe = pd.DataFrame(R,columns=['R'])
dataframe["G"] = G
dataframe["B"] = B

dataframe.dtypes
for col in dataframe:
    dataframe[col] = pd.to_numeric(dataframe[col], errors='coerce')
    
dataframe = dataframe.interpolate(method ='linear', limit_direction ='forward')   # Recreate  the data with respect to previous data
R = dataframe['R']
G = dataframe['G']
B = dataframe['B']

In [ ]:
plt.plot(R)

In [ ]:
import scipy.interpolate as intp

R = np.array(R)   # using another interpolation we increase the data size
G = np.array(G)
B = np.array(B)

R_new_intp = intp.interp1d(np.arange(R.size),R)
R1 = R_new_intp(np.linspace(0, R.size-1, 3816))

G_new_intp = intp.interp1d(np.arange(G.size),G)
G1 = G_new_intp(np.linspace(0, G.size-1, 3816))

B_new_intp = intp.interp1d(np.arange(B.size),B)
B1 = B_new_intp(np.linspace(0, B.size-1, 3816))

In [ ]:
R = np.array(R)
G = np.array(G)
B = np.array(B)   # Using another intepolation we at first decrease the size & next increase the size

R_intp = intp.interp1d(np.arange(R.size),R)
R_new = R_intp(np.linspace(0, R.size-1, 1700))

R_new_intp = intp.interp1d(np.arange(R_new.size),R_new)
R1 = R_new_intp(np.linspace(0, R_new.size-1, 2072))
#G

G_intp = intp.interp1d(np.arange(G.size),G)
G_new = G_intp(np.linspace(0, G.size-1, 1700))

G_new_intp = intp.interp1d(np.arange(G_new.size),G_new)
G1 = G_new_intp(np.linspace(0, G_new.size-1, 2072))
#B

B_intp = intp.interp1d(np.arange(B.size),B)
B_new = B_intp(np.linspace(0, B.size-1, 1700))

B_new_intp = intp.interp1d(np.arange(B_new.size),B_new)
B1 = B_new_intp(np.linspace(0, B_new.size-1, 2072))

In [ ]:
# nR = (resample_R - np.mean(resample_R)) #/np.var(R)
# nG = (resample_G - np.mean(resample_G)) #/np.var(G)
# nB = (resample_B - np.mean(resample_B)) #/np.var(B)

# nR = (R1 - np.mean(R1)) #/np.var(R)
# nG = (G1 - np.mean(G1)) #/np.var(G)
# nB = (B1 - np.mean(B1)) #/np.var(B)

nR = (R - np.mean(R)) #/np.var(R)
nG = (G - np.mean(G)) #/np.var(G)
nB = (B - np.mean(B)) #/np.var(B)

plt.figure()
plt.subplot(3,1,1)
plt.plot(nR,color='red')
plt.subplot(3,1,2)
plt.plot(nG,color= 'Green')
plt.subplot(3,1,3)
plt.plot(nB, color='blue')

In [ ]:
from scipy import signal

plt.figure()
plt.subplot(3,1,1)
plt.plot(signal.detrend(nR),color='red')
plt.subplot(3,1,2)
plt.plot(signal.detrend(nG),color= 'Green')
plt.subplot(3,1,3)
plt.plot(signal.detrend(nB), color='blue')

In [ ]:
nR = signal.detrend(nR)
nG = signal.detrend(nG)
nB = signal.detrend(nB)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

x = nR
y1 = running_mean(x, 10)
plt.subplot(1,2,1)
plt.plot(x)
plt.subplot(1,2,2)
plt.plot(y1,'r')

In [ ]:
def center(x):
    mean = np.mean(x, axis=1, keepdims=True)
    centered =  x - mean
    return centered, mean

def covariance(x):
    mean = np.mean(x, axis=1, keepdims=True)
    n = np.shape(x)[1] - 1
    m = x - mean

    return (m.dot(m.T))/n

def whiten(x):
    # Calculate the covariance matrix
    coVarM = covariance(X)

    # Single value decoposition
    U, S, V = np.linalg.svd(coVarM)

    # Calculate diagonal matrix of eigenvalues
    d = np.diag(1.0 / np.sqrt(S))

    # Calculate whitening matrix
    whiteM = np.dot(U, np.dot(d, U.T))

    # Project onto whitening matrix
    Xw = np.dot(whiteM, X)

    return Xw, whiteM

def fastIca(signals,  alpha = 1, thresh=1e-8, iterations=5000):
    m, n = signals.shape

    # Initialize random weights
    W = np.random.rand(m, m)

    for c in range(m):
            w = W[c, :].copy().reshape(m, 1)
            w = w / np.sqrt((w ** 2).sum())

            i = 0
            lim = 100
            while ((lim > thresh) & (i < iterations)):

                # Dot product of weight and signal
                ws = np.dot(w.T, signals)

                # Pass w*s into contrast function g
                wg = np.tanh(ws * alpha).T

                # Pass w*s into g prime
                wg_ = (1 - np.square(np.tanh(ws))) * alpha

                # Update weights
                wNew = (signals * wg.T).mean(axis=1) - wg_.mean() * w.squeeze()

                # Decorrelate weights              
                wNew = wNew - np.dot(np.dot(wNew, W[:c].T), W[:c])
                wNew = wNew / np.sqrt((wNew ** 2).sum())

                # Calculate limit condition
                lim = np.abs(np.abs((wNew * w).sum()) - 1)

                # Update weights
                w = wNew

                # Update counter
                i += 1

            W[c, :] = w.T
    return W

In [ ]:
X = (np.c_[nR, nG, nB]).T

In [ ]:
# Center signals
Xc, meanX = center(X)

# Whiten mixed signals
Xw, whiteM = whiten(Xc)

# Check if covariance of whitened matrix equals identity matrix
print(np.round(covariance(Xw)))

In [ ]:
W = fastIca(Xw,  alpha=1)

#Un-mix signals using
unMixed = Xw.T.dot(W.T)

# Subtract mean
unMixed = (unMixed.T - meanX).T

In [ ]:

colors = ['red', 'green', 'blue']
fig, ax = plt.subplots(1, 1, figsize=[18, 5])
for i in range(3):
    ax.plot(X[i], lw = 2, color = colors[i])
ax.tick_params(labelsize=12)
ax.set_title('mixed signals', fontsize=25)
# ax.set_xlim(0, 10)

fig, ax = plt.subplots(1, 1, figsize=[18, 5],)
for i in range(3):
    ax.plot(unMixed.T[i], lw = 2, color = colors[i])
ax.set_ylabel("Amplitude", fontsize = 10)
ax.set_xlabel('Sample number', fontsize=20)
ax.set_title('Recovered signals', fontsize=25)
# ax.set_xlim(0, 100)

plt.show()

In [ ]:
ff = plt.figure()
ff.set_figwidth(30)
ff.set_figheight(8)
plt.plot(unMixed.T[2])

In [ ]:
pip install EMD-signal

In [ ]:
from PyEMD import EEMD

In [ ]:
from PyEMD import  Visualisation

In [ ]:
pip install vmdpy

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from numpy import corrcoef as cc
from scipy import signal
from scipy.fft import fft, fftfreq
from scipy.stats import kurtosis
from scipy.stats import skew
from PyEMD import EEMD, Visualisation
from vmdpy import VMD
import matplotlib.pyplot as plt
import cmath
from datetime import datetime

In [ ]:
fs = 18
sos_resp = signal.butter(2, [0.15, 0.65], 'bandpass', output='sos', fs=fs)
sos_heart = signal.butter(2, [0.75, 5.0], 'bandpass', output='sos', fs=fs)

In [ ]:
filt_resp = signal.sosfiltfilt(sos_resp, unMixed.T[2])
filt_heart= signal.sosfiltfilt(sos_heart, unMixed.T[2])
N = len(unMixed.T[2])
ff_filtresp = np.abs(fft(filt_resp,N))
ff_filtheart = np.abs(fft(filt_heart,N))

In [ ]:
peaks_index,_ = signal.find_peaks(ff_filtresp[:60])
peaks = ff_filtresp[peaks_index]
freqs = fftfreq(len(filt_resp)) * fs
peak_freq =freqs[peaks_index]
print(f'Peaks: {peaks}')
print(f'Frequency values at respiration peaks: {peak_freq}')
peaks_index11,_ = signal.find_peaks(ff_filtheart[:60])
peaks11 = ff_filtheart[peaks_index11]
freqs11 = fftfreq(len(filt_heart)) * fs
peak_freq11 =freqs[peaks_index11]
print(f'Peaks: {peaks11}')
print(f'Frequency values at heart peaks: {peak_freq11}')

In [ ]:
plt.figure(figsize=(15,4))
plt.plot(filt_resp, 'b')
plt.title('Respiration Signal')
plt.grid()


plt.figure(figsize=(15,4))
plt.plot(filt_heart, 'r')
plt.title('Heart Signal')
plt.grid()
       
plt.show()
      


In [ ]:
eemd=EEMD()
IMFss = eemd(filt_resp)
imfNo  = IMFss.shape[0]
print("Total Number of IMFs in resp signal = " + str(imfNo))

imfs, res = IMFss[:-1], IMFss[-1]

In [ ]:
eemd1=EEMD()
IMFss1 = eemd(filt_heart)
imfNo1  = IMFss1.shape[0]
print("Total Number of IMFs in heart signal= " + str(imfNo1))

imfs1, res1 = IMFss1[:-1], IMFss[-1]

In [ ]:
cnt = 0

plt.figure(figsize=(25, 25)) 
plt.subplots_adjust(hspace=0.4)

R=[]

for i in imfs:
    
    
    plt.subplot(len(imfs), 1, cnt+1)
    plt.suptitle("Freq response of EEMD_RESP, IPPG",y=0.9)
    t=30
    r1=np.abs(fft(i,N))[:t]
    R.append(r1)
    plt.plot(freqs[0:t],10*r1,'k')
    
    plt.grid()
    
    cnt += 1


In [ ]:
cnt = 0

plt.figure(figsize=(25, 25)) 
plt.subplots_adjust(hspace=0.4)

R1=[]

for i in imfs1:
    
    
    plt.subplot(len(imfs1), 1, cnt+1)
    plt.suptitle("Freq response of EEMD_HEART, IPPG",y=0.9)
    t=100
    r2=np.abs(fft(i,N))[:t]
    R1.append(r2)
    plt.plot(freqs11[0:t],10*r2,'b')
    
    plt.grid()
    
    cnt += 1


In [ ]:
HR_sig1=IMFss[3]
ff_RR1=np.abs(fft(HR_sig1,N))*5

peaks_index1,_= signal.find_peaks(ff_RR1[0:160])
peaks1=ff_RR1[peaks_index1]
max_peak1=max(peaks1)
index1=np.where(peaks1==max_peak1)
max_peak_index1=peaks_index1[index1]
RR_freq1=freqs[max_peak_index1]
HR_sig=HR_sig1
RR_freq=RR_freq1
ff_RR=ff_RR1
ind=max_peak_index1
max_peak=max_peak1
peak_index=peaks_index1
peaks=peaks1
index=index1

print(RR_freq*60)


In [ ]:
HR_sig11=IMFss1[1]
ff_RR2=np.abs(fft(HR_sig11,N))*5

peaks_index10,_= signal.find_peaks(ff_RR2[0:160])
peaks10=ff_RR2[peaks_index10]
max_peak10=max(peaks10)
index10=np.where(peaks10==max_peak10)
max_peak_index10=peaks_index10[index10]
RR_freq10=freqs[max_peak_index10]
HR_sig=HR_sig11
RR_freq100=RR_freq10
ff_RR10=ff_RR2
ind=max_peak_index10
max_peak100=max_peak10
peak_index100=peaks_index10
peaks=peaks10
index=index10
print(RR_freq10*60)
